In [1]:
import pandahouse as ph

In [2]:
connection_def_DB = {'host': 'https://clickhouse.lab.karpov.courses',
                         'database': 'def',
                         'user': 'admin',
                         'password': 'admin'}

In [3]:
def return_data(query):
    return ph.read_clickhouse(query,
                              connection=connection_def_DB)

### 1. Очень усердные ученики

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц <span style="color:Tomato">**правильно**</span> решил 20 горошин.

Необходимо написать <span style="color:Tomato">**оптимальный запрос**</span>, который даст информацию о количестве очень усердных студентов.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

#### <span style="color:DarkGreen">Поскольку в нашей таблице только один месяц, то в запросе можно не прописывать конкретное указание на "текущий месяц".</span>

In [4]:
query = """
            SELECT
                countIf(qty_answ >= 20) as qnty_hard_students
            FROM
                (
                SELECT
                    st_id, COUNT(correct) as qty_answ
                FROM
                    peas
                WHERE
                    correct == 1 AND MONTH(timest) == 10
                GROUP BY
                    st_id
                ) as act_stud
        """

In [5]:
return_data(query)

,qnty_hard_students
0,136


### 2. Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Даны таблицы: default.peas, default.studs и default.final_project_check

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- CR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

<span style="color:Tomato">**Активным**</span> считается пользователь, за все время решивший <span style="color:Tomato">**больше**</span> 10 задач правильно в любых дисциплинах.

<span style="color:Tomato">**Активным**</span> по математике считается пользователь, за все время решивший <span style="color:Tomato">**2 или больше**</span> задач правильно по математике.

In [6]:
query_2 = """
            SELECT
                test_grp as Group,
                ROUND(SUM(revenue) / COUNT(peas.st_id), 0) as ARPU,
                ROUND(sumIf(revenue, correct_answr > 10) /
                countIf(peas.st_id, correct_answr > 10), 0) as ARPAU,
                CONCAT(CAST(ROUND((countIf(peas.st_id, revenue > 0) /
                COUNT(peas.st_id) * 100), 2) AS String), '%') AS CR,
                CONCAT(CAST(ROUND((countIf(peas.st_id, (revenue > 0 AND correct_answr > 10)) /
                countIf(peas.st_id, correct_answr > 10)) * 100, 2) AS String), '%') as CR_active_users,
                CONCAT(CAST(ROUND((countIf(peas.st_id, (revenue_math > 0 AND corr_math  >= 2)) /
                countIf(peas.st_id, corr_math >= 2)) * 100, 2) AS String), '%') as CR_math
            FROM
                (
                SELECT
                    peas.st_id,
                    test_grp,
                    correct_answr,
                    corr_math,
                    revenue,
                    revenue_math
                FROM default.studs as studs
                    LEFT JOIN
                    (
                        SELECT
                            st_id,
                            SUM(correct) as correct_answr,
                            sumIf(correct, subject = 'Math') as corr_math
                        FROM default.peas
                        GROUP BY st_id
                    ) as peas
                    ON studs.st_id = peas.st_id
                    LEFT JOIN
                    (
                        SELECT
                            st_id,
                            SUM(money) as revenue,
                            sumIf(money, subject = 'Math') as revenue_math
                        FROM default.final_project_check
                        GROUP BY st_id
                    ) as final
                    ON studs.st_id = final.st_id
                )
            GROUP BY
                test_grp
        """

In [7]:
return_data(query_2)

,Group,ARPU,ARPAU,CR,CR_active_users,CR_math
0,control,4541.0,10394.0,4.92%,11.02%,6.12%
1,pilot,11508.0,29740.0,10.85%,26.04%,9.52%
